In [1]:
from scapy.all import *
import os
import re
import base64
import requests
import ipdb
import json
import re

In [45]:
def parseDnsPcap(filename, output):
    """
    使用thshark将pcap解析为json数据
    """
    outputPath = os.path.join(output, "{}.json".format(filename.split('/')[-1]))
    cmd = 'tshark -r {} -Y "(dns.resp.type==16 || dns.resp.type==1) && dns.qry.name contains \"ssltestdomain.xyz\"" -T json -e frame.time -e dns.qry.type -e dns.qry.name -e dns.resp.name -e dns.txt >> {}'.format(filename, outputPath)
    print(cmd)
    os.system(cmd)


In [117]:
parseDnsPcap('../../dns-order/dns_sample/file_explore.pcap', '../../dns-order/dns_sample/')
parseDnsPcap('../../dns-order/dns_sample/hashdump.pcap', '../../dns-order/dns_sample/')
parseDnsPcap('../../dns-order/dns_sample/screenshot.pcap', '../../dns-order/dns_sample/')
parseDnsPcap('../../dns-order/dns_sample/shell_whoami.pcap', '../../dns-order/dns_sample/')
parseDnsPcap('../../dns-order/dns_sample/sleep_10.pcap', '../../dns-order/dns_sample/')
parseDnsPcap('../../dns-order/dns.pcap', '../../dns-order/')

tshark -r ../../dns-order/dns_sample/file_explore.pcap -Y "(dns.resp.type==16 || dns.resp.type==1) && dns.qry.name contains "ssltestdomain.xyz"" -T json -e frame.time -e dns.qry.type -e dns.qry.name -e dns.resp.name -e dns.txt >> ../../dns-order/dns_sample/file_explore.pcap.json
tshark -r ../../dns-order/dns_sample/hashdump.pcap -Y "(dns.resp.type==16 || dns.resp.type==1) && dns.qry.name contains "ssltestdomain.xyz"" -T json -e frame.time -e dns.qry.type -e dns.qry.name -e dns.resp.name -e dns.txt >> ../../dns-order/dns_sample/hashdump.pcap.json
tshark -r ../../dns-order/dns_sample/screenshot.pcap -Y "(dns.resp.type==16 || dns.resp.type==1) && dns.qry.name contains "ssltestdomain.xyz"" -T json -e frame.time -e dns.qry.type -e dns.qry.name -e dns.resp.name -e dns.txt >> ../../dns-order/dns_sample/screenshot.pcap.json
tshark -r ../../dns-order/dns_sample/shell_whoami.pcap -Y "(dns.resp.type==16 || dns.resp.type==1) && dns.qry.name contains "ssltestdomain.xyz"" -T json -e frame.time -e dn

In [114]:
with open('../../dns-order/dns.pcap.json', 'r') as f:
    shell_whoami_data = json.loads(f.read())
#     shell_whoami_data = [x['_source']['layers'] for x in shell_whoami_data if x['_source']['layers']['dns.qry.type'][0] == '1' and x['_source']['layers']['dns.qry.name'][0].startswith('post')]
#     shell_whoami_data = [x['_source']['layers'] for x in shell_whoami_data if x['_source']['layers']['dns.qry.type'][0] == '16']
    shell_whoami_data = [x['_source']['layers'] for x in shell_whoami_data]
shell_whoami_data = sorted(shell_whoami_data, key = lambda x: x['frame.time'][0])
with open('dns_pcap.txt', 'w') as f:
    for item in shell_whoami_data:
        for key, value in item.items():
            print("{}: {}".format(key, value), file=f)
        print(file=f)

In [129]:
def parseTxtDnsData(filename, showData=True):
    with open(filename, 'r') as f:
        dns_data = json.loads(f.read())
        dns_data = [x['_source']['layers'] for x in dns_data if x['_source']['layers']['dns.qry.type'][0] == '16']
        dns_data = sorted(dns_data, key = lambda x: x['frame.time'][0])
    allIndexs = []
    for item in dns_data:
        tempIndex = item['dns.qry.name'][0].split('.')[1][-5:]
        if tempIndex in allIndexs:
#             print(tempIndex)
            continue
        else:
            allIndexs.append(tempIndex)
    result = []
    for index in allIndexs:
        seriesData = []
        seriesNumberSet = []
        for item in dns_data:
            item_index = item['dns.qry.name'][0].split('.')[1][-5:]
            if item_index != index:
                continue
            seriesNumber = item['dns.qry.name'][0].split('.')[1][:-5]
            if seriesNumber in seriesNumberSet:
                continue
            else:
                seriesNumberSet.append(seriesNumber)
                item['series'] = int(seriesNumber, 16)
                seriesData.append(item)
        seriesData = sorted(seriesData, key=lambda x: x['series'])
        indexResult = {}
        txtData = ""
        for item in seriesData:
            txtData += item['dns.txt'][0]
        if showData:
            indexResult['data'] = txtData
        indexResult['payloadLen'] = len(txtData)
        indexResult['time'] = seriesData[0]['frame.time'][0]
        indexResult['dns.qry.name.0'] = seriesData[0]['dns.qry.name'][0]
        indexResult['totalNum'] = len(seriesData)
        indexResult['index'] = index
        indexResult['type'] = "TXT"
        result.append(indexResult)
    return result
parseTxtDnsData('../../dns-order/dns_sample/shell_whoami.pcap.json', showData=True)

[{'data': 'YlbbjzgG/cladUe05fZZq0E5G+gJQqUJ+GPprP3sN9Q5E/Zl34OU94qbKA1kmmrhXgNxyeYVFe49CEjT3qJJdQ==',
  'payloadLen': 88,
  'time': 'Sep  3, 2021 15:52:44.240984000 CST',
  'dns.qry.name.0': 'api.1376029d5.2f195fb2.ns1.ssltestdomain.xyz',
  'totalNum': 1,
  'index': '6029d5',
  'type': 'TXT'}]

In [65]:
with open('../../dns-order/dns_sample/shell_whoami.pcap.json', 'r') as f:
    shell_whoami_data = json.loads(f.read())
    shell_whoami_data = [x['_source']['layers'] for x in shell_whoami_data if x['_source']['layers']['dns.qry.type'][0] == '1' and x['_source']['layers']['dns.qry.name'][0].startswith('www')]
#     shell_whoami_data = [x['_source']['layers'] for x in shell_whoami_data if x['_source']['layers']['dns.qry.type'][0] == '16']
shell_whoami_data = sorted(shell_whoami_data, key = lambda x: x['frame.time'][0])
for item in shell_whoami_data:
    print(item)

{'frame.time': ['Sep  3, 2021 15:52:22.038562000 CST'], 'dns.qry.type': ['1'], 'dns.qry.name': ['www.180.03fd576a2.2f195fb2.ns1.ssltestdomain.xyz'], 'dns.resp.name': ['www.180.03fd576a2.2f195fb2.ns1.ssltestdomain.xyz']}
{'frame.time': ['Sep  3, 2021 15:52:22.268154000 CST'], 'dns.qry.type': ['1'], 'dns.qry.name': ['www.36d1b299763f37592ee17fea0b653aab7fc21871a417a1bb72ff7adb8.d175fcee359f6f862fc0fcc9a709cede8dbb3877b29279de0e1f5a82.9fea09676f15230a8fedfc9c8356c11136e12829499daafdecf6fd5e.13fd576a2.2f195fb2.ns1.ssltestdomain.xyz'], 'dns.resp.name': ['www.36d1b299763f37592ee17fea0b653aab7fc21871a417a1bb72ff7adb8.d175fcee359f6f862fc0fcc9a709cede8dbb3877b29279de0e1f5a82.9fea09676f15230a8fedfc9c8356c11136e12829499daafdecf6fd5e.13fd576a2.2f195fb2.ns1.ssltestdomain.xyz']}
{'frame.time': ['Sep  3, 2021 15:52:23.241129000 CST'], 'dns.qry.type': ['1'], 'dns.qry.name': ['www.2f2ff8227173e648f460cdbeaa06376d3b72b77cd.0d0017817c39ed603052f5fe2ed2a139294c0cde.23fd576a2.2f195fb2.ns1.ssltestdomain.xyz

In [73]:
def parseADnsData(filename, showData=True):
    with open(filename, 'r') as f:
        dns_data = json.loads(f.read())
        dns_data = [x['_source']['layers'] for x in dns_data if x['_source']['layers']['dns.qry.type'][0] == '1'and (x['_source']['layers']['dns.qry.name'][0].startswith('post'))]
#         dns_data = [x['_source']['layers'] for x in dns_data if x['_source']['layers']['dns.qry.type'][0] == '1']
        dns_data = sorted(dns_data, key = lambda x: x['frame.time'][0])
    allIndexs = []
    for item in dns_data:
#         print(item['dns.qry.name'][0])
        tempIndex = item['dns.qry.name'][0].split('.')[-5][-5:]
        if tempIndex in allIndexs:
            continue
        else:
            allIndexs.append(tempIndex)
    result = []
    for index in allIndexs:
        seriesData = []
        seriesNumberSet = []
        for item in dns_data:
            item_index = item['dns.qry.name'][0].split('.')[-5][-5:]
            if item_index != index:
                continue
            seriesNumber = item['dns.qry.name'][0].split('.')[-5][:-5]
            if seriesNumber in seriesNumberSet:
                continue
            else:
                seriesNumberSet.append(seriesNumber)
                item['series'] = int(seriesNumber, 16)
                seriesData.append(item)
        seriesData = sorted(seriesData, key=lambda x: x['series'])
        indexResult = {}
        txtData = ""
        for item in seriesData:
            qryNameSplits = item['dns.qry.name'][0].split('.')
            if len(qryNameSplits) == 7:
                txtData += qryNameSplits[1]
            else:
                txtData += "".join(qryNameSplits[1:3])
        if showData:
            indexResult['data'] = txtData
        indexResult['payloadLen'] = len(txtData)
        indexResult['time'] = seriesData[0]['frame.time'][0]
        indexResult['dns.qry.name.0'] = seriesData[0]['dns.qry.name'][0]
        indexResult['totalNum'] = len(seriesData)
        indexResult['index'] = index
        indexResult['type'] = "A"
        result.append(indexResult)
    return result
len(parseADnsData('../../dns-order/dns.pcap.json', showData=True))

36

In [75]:
samplePath = "../../dns-order/dns_sample/"
files = [os.path.join(samplePath, x) for x in os.listdir(samplePath) if 'json' in x]
sampleResult = {}
for file in files:
    temp = []
    temp += parseTxtDnsData(file)
    temp += parseADnsData(file)
    sampleResult[file.split('/')[-1]] = temp
# sampleResult = sorted(sampleResult, key=lambda x: x['time'])
for key, value in sampleResult.items():
    print()
    print(key)
    value = sorted(value, key=lambda x: x['time'])
    for item in value:
        print("type: {}, total: {}".format(item['type'], item['payloadLen']))


hashdump.pcap.json
type: TXT, total: 110040
type: A, total: 310

screenshot.pcap.json
type: TXT, total: 216536
type: A, total: 215676

shell_whoami.pcap.json
type: TXT, total: 88
type: A, total: 133

file_explore.pcap.json
type: TXT, total: 88
type: A, total: 456
type: A, total: 100
type: TXT, total: 108
type: A, total: 763
type: A, total: 763
type: TXT, total: 88
type: A, total: 828

sleep_10.pcap.json
type: TXT, total: 88


In [130]:
result = []
result += parseTxtDnsData('../../dns-order/dns.pcap.json', showData=True)
result += parseADnsData('../../dns-order/dns.pcap.json', showData=True)
num = 0
for item in result:
    num += 1
num

ValueError: invalid literal for int() with base 16: ''

In [119]:
result = sorted(result, key=lambda x: x['time'])
with open('./dnsAllDataWithData.txt', 'w') as f:
    for item in result:
#         if item['valid']:
        for key, value in item.items():
            print("{}: {}".format(key, value), file=f)
        print(file=f)

In [238]:
joinTxtDnsData('../../dns-order/dns.pcap.json')

[{'responseData': 'RAwSvQ+3hEl3WpnF8eQGfCOhMYhnYRV/aO45pzbZm3NPzCIpqTLVhj6P4a4EsJA8WATbBeRo6PjIdpOleo2dcA==zHHcHK5rw15m/BrBf5/vHSJZOun9La8BrH2b7uVX2wQbPSs5zgafuMlNIK/4j3844FhRiER+xs3PoYrD23MarQ==RzpGa7zDuUv11Ixp71UAexiV2dKSU5Np8vyPLdcSBngEMf8dWgo5hTFXSjpKyyqWzTzRw1VbA9cGWxN4hd/1bA==rZq/+Hxwhn5mpxFnzTb9BvPAYr9g1KIiMuiaMus6R0mB9pReNAm+Z1I0cVHoyXXJ2/v0CEKsdXJVkCYac0L+fA==aFaW/88Q0a48vbz4Eqa+OyOUBgQ2L0ykhfpNH+OmYXAGUiLLUKnjd9FF4BiPV+QibvSyMrKRpbyUEik7jRSdjkazlpiSMJBK/EpjwDoNXtYwZD7m8Nompld6oKaQZNqAeZACooD9AyXcyWGdpFnycecaYWwNOxgxVLKmhWD0dJ0=Yu0H/B1IfLntHUpN5Oo7S89IhQU1DqClv1ffTsPb58zxVXEZXphjdVaCPX4pAFcFFVRJwqjSQFp54tm0SackgA==Q74ZkmWg/nUzAvA0Y0hz+g4WVpgE0Y0dZkSDY1YzPK4rfSjl5w+6usJIEoG7UhJa0M6VvfHq7VZmE/QqF61F4A==duSTMlyhcd8w5ygyTR3Z6XVxv5Pb51aFBm78yz3ecG0R24XnC8Vs27gUnROwywHkjGY8V75e/VKrwFYtTyV1Fl2lugkZ9RqoTtu1Rk9mAfStJUch4u0U9/pYhK4h4EEC95YGuTYHUmOq3oOiAe2asfLEzOJ+NhbGpHB+3SoxEAM9F7f0pGt3DxGCQ6dXt+oOgCaTFo+rOMA1XLq1PVgMMYIr5VwQvTfNEchZdUsSSOU=9nNHmVMJcwXzNz67TkUBF10Hvhk3NnMjhHfDNjwjQvWXS

In [64]:
# with open('dnsData.json', 'w') as f:
#     result_json = json.dumps(result)
#     f.write(result_json)


In [100]:
# respdata = result[7]['responseData']
# decodeData = base64.b64decode(respdata)
# with open('test.png', 'wb') as f:
#     f.write(decodeData)

In [87]:
print(len(result))

49


In [103]:
result = sorted(result, key=lambda x: x['time'])
f = open('./dnsData.txt', 'w')
for item in result:
    for key, value in item.items():
        print("{}: {}".format(key, value), file=f)
    print(file=f)
    

In [104]:
for item in result:
    if item['index'] == '54126306':
        for key, value in item.items():
            print("{}: {}".format(key, value))

responseData: ZQOxjNdQENqo5isVYcQ3QSoGETrEOv8/ZNyFihVhj2uSE7yLlEsznbp3M6IOc0KCIDBI60WBQC+uS24vc8KcSQ==
time: Sep 28, 2021 18:21:04.443235000 CST
dns.qry.name: api.154126306.2bb0a7b2.ns1.ssltestdomain.xyz
totalBytes: 88
index: 54126306
totalNum: 1


In [108]:
for key, value in sampleResult.items():
    print(key)
    for item in value:
        for key_, value_ in item.items():
            print("{}: {}".format(key_, value_))

hashdump.pcap.json
responseData: ...
time: Sep  3, 2021 15:53:27.001210000 CST
dns.qry.name: api.16d866172.2f195fb2.ns1.ssltestdomain.xyz
totalBytes: 110040
index: 6d866172
totalNum: 437
screenshot.pcap.json
responseData: ...
time: Sep  3, 2021 16:07:48.172594000 CST
dns.qry.name: api.157ce111a.782f55f2.ns1.ssltestdomain.xyz
totalBytes: 216536
index: 57ce111a
totalNum: 860
shell_whoami.pcap.json
responseData: YlbbjzgG/cladUe05fZZq0E5G+gJQqUJ+GPprP3sN9Q5E/Zl34OU94qbKA1kmmrhXgNxyeYVFe49CEjT3qJJdQ==
time: Sep  3, 2021 15:52:44.240984000 CST
dns.qry.name: api.1376029d5.2f195fb2.ns1.ssltestdomain.xyz
totalBytes: 88
index: 376029d5
totalNum: 1
file_explore.pcap.json
responseData: 8lcerYZ6scQKYy0iUOMF+dhvxaxVMSGIcpSqkwCmET5UCUdBcQnUsJbDraSxvWXo9za/DI51g0a9Pb6lRdq7MA==
time: Sep  3, 2021 16:03:09.754526000 CST
dns.qry.name: api.17f994c78.782f55f2.ns2.ssltestdomain.xyz
totalBytes: 88
index: 7f994c78
totalNum: 1
responseData: 9gKAs0LKBBhuozleU4MxDW4bx+3pRRHyvOfp81ZBvMJu7FlqjoWM6snt/XsbzzeBh/kgPc

In [117]:
len(sampleResult['file_explore.pcap.json'])

3

In [112]:
sampleResult.keys()

dict_keys(['hashdump.pcap.json', 'screenshot.pcap.json', 'shell_whoami.pcap.json', 'file_explore.pcap.json', 'sleep_10.pcap.json'])

In [118]:
for item in sampleResult['file_explore.pcap.json']:
    print(item['totalBytes'])

88
108
88


In [119]:
sampleResult['file_explore.pcap.json'][0]

{'responseData': '8lcerYZ6scQKYy0iUOMF+dhvxaxVMSGIcpSqkwCmET5UCUdBcQnUsJbDraSxvWXo9za/DI51g0a9Pb6lRdq7MA==',
 'time': 'Sep  3, 2021 16:03:09.754526000 CST',
 'dns.qry.name': 'api.17f994c78.782f55f2.ns2.ssltestdomain.xyz',
 'totalBytes': 88,
 'index': '7f994c78',
 'totalNum': 1}

In [125]:
ss = "LeXxPBGiUnVJwJOHEoJCxhN6Wa/GgO7Voo/oVGCTWWgzNuFOYqr8hb6e7xIaQeXy/VPWSUSjAkVW7pUEt8PW7Q=="
de_ss = base64.b64decode(ss)
for item in de_ss:
    print(chr(item))

-
å
ñ
<

¢
R
u
I
À




B
Æ

z
Y
¯
Æ

î
Õ
¢

è
T
`

Y
h
3
6
á
N
b
ª
ü

¾

ï


A
å
ò
ý
S
Ö
I
D
£

E
V
î


·
Ã
Ö
í
